In [1]:
"""Notebook illustrates reduction of variance in case of stratified sampling."""
import numpy as np
from tqdm.notebook import tqdm

In [2]:
def make_syntetic_general_dist(len_first=10**7, len_second=10**6):
  """Generate two syntetic general population.

  :param len_first: size of first population
  :param len_second: size of second population
  """
  population_strata_first = np.random.normal(100, 10, len_first)
  population_strata_second = np.random.normal(50, 5, len_second)
  return population_strata_first, population_strata_second


def make_groups_before_after_treatment(population_total, len_sampled, len_strata_first):
  """Sampling groups for before and after from population.

  :param population_total: data(np.array()) before treatment from population
  :param len_sampled: size of sample
  :param len_strata_first: size of first strata in popultaion
  """
  # syntetic group from population before treatment
  indexes = np.random.choice(range(len(population_total)), len_sampled, False)
  strat_1_before = population_total[indexes[indexes <= len_strata_first]]
  strat_2_before = population_total[indexes[indexes > len_strata_first]] 

  # syntetic group from population before treatment
  indexes = np.random.choice(range(len(population_total)), len_sampled, False)
  strat_1_after = population_total[indexes[indexes <= len_strata_first]]
  strat_2_after = population_total[indexes[indexes > len_strata_first]] 
  return strat_1_before, strat_2_before, strat_1_after, strat_2_after


def count_strat_se(sampled_after, strat_1_after, sample_len, w1, w2):
  """Count strat mean and se.

  :param sampled after: data(np.array()) after treatment
  :param strat_1_after: data(np.array()) after treatment from first strata
  :param sample_len: size of sample
  :param w1: weight of first strata
  :param w2: weight of second strata
  """
  indexes = np.random.choice(range(len(sampled_after)), sample_len, False)
  sample_strat_1 = sampled_after[indexes[indexes <= len(strat_1_after)]] 
  sample_strat_2 = sampled_after[indexes[indexes > len(strat_1_after)]]
  var_with_strat = (np.var(sample_strat_1) * w1 + np.var(sample_strat_2) * w2) / sample_len
  strat_mean = w1 * np.mean(sample_strat_1) + w2 * np.mean(sample_strat_2)
  strat_se = np.sqrt(var_with_strat)
  return strat_mean, strat_se

In [5]:
def make_experiment(len_of_sample, n_iters=100):
  """Experiment strat sampling vs global sampling.

  :param len_of_sample: size of sample in experiment
  :param n_iters: number of iterations in experiment
  """
  mean_strat_list = []
  se_strat_list = []
  for _ in tqdm(range(n_iters)):
    population_strata_first, population_strata_second = make_syntetic_general_dist()
    len_sampled = int(0.1 * (len(population_strata_first) + len(population_strata_second)))

    population_total = np.concatenate((population_strata_first, population_strata_second))
    global_mean = np.mean(population_total)

    strat_1_before, strat_2_before, strat_1_after, strat_2_after = make_groups_before_after_treatment(
        population_total, len_sampled, len(population_strata_first)
    )

    sampled_before = np.concatenate((strat_1_before, strat_2_before))
    sampled_after = np.concatenate((strat_1_after, strat_2_after))

    w1 = len(strat_1_before) / len(sampled_before) 
    w2 = len(strat_2_before) / len(sampled_before)

    strat_mean, strat_se = count_strat_se(sampled_after, strat_1_after, len_of_sample, w1, w2)
    mean_strat_list.append(strat_mean)
    se_strat_list.append(strat_se)

  print('se of strat mean: ', round(np.std(mean_strat_list), 3))
  print('se of strat mean in theory: ', round(strat_se, 3))

  means = []
  for _ in range(n_iters ** 2):
      sampled = np.random.choice(sampled_after, len_of_sample, True)
      means.append(np.mean(sampled))
  print('se of global mean in bootstrapping: ', round(np.std(means), 3))
  print('se of global mean in theory: ', round(np.sqrt(np.var(sampled_after) / len_of_sample), 3))

In [6]:
make_experiment(10000)

  0%|          | 0/100 [00:00<?, ?it/s]

se of strat mean:  0.097
se of strat mean in theory:  0.095
se of global mean in bootstrapping:  0.172
se of global mean in theory:  0.173


In [7]:
make_experiment(100000)

  0%|          | 0/100 [00:00<?, ?it/s]

se of strat mean:  0.032
se of strat mean in theory:  0.031
se of global mean in bootstrapping:  0.054
se of global mean in theory:  0.055


In [8]:
make_experiment(150000)

  0%|          | 0/100 [00:00<?, ?it/s]

se of strat mean:  0.028
se of strat mean in theory:  0.025
se of global mean in bootstrapping:  0.045
se of global mean in theory:  0.045
